<a href="https://colab.research.google.com/github/shadowjljp/Machine_learning/blob/master/K_nearest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#load CSV file
from csv import reader
#import data from google drive
from google.colab import drive
drive.mount('/content/gdrive')

from statistics import mean 
# k-nearest neighbors on the Iris Flowers Dataset
from random import seed
from random import randrange
from csv import reader
from math import sqrt
# Example of calculating Euclidean distance
from math import sqrt

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for _ in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())
 
# Convert string column to integer
def str_column_to_int(dataset, column):
  class_values = [row[column] for row in dataset]
  unique = set(class_values)
  lookup = dict()
  for i, value in enumerate(unique):
    lookup[value] = i
  for row in dataset:
    row[column] = lookup[row[column]]
  return lookup
 # Find the min and max values for each column
def dataset_minmax(dataset):
  minMax=[]
  for i in range (len(dataset)):
    dataCol = [row[i] for row in dataset]
    valueMin = min(dataCol)
    valueMax = max(dataCol)
    valueMean = mean(dataCol)
    minMax.append([valueMin,valueMax,valueMean])
  return minMax

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
  for row in dataset:
    for i in range (len(row)):
      row[i]=(row[i]-minmax[2])/(minmax[1]-minmax[0])

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores
# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0
  
# calculate Euclidean Distance
def EuclideanDistance(row1,row2):
  distance=0.0
  for i in range(len(row1)-1):
    distance+=(row1[i]-row2[i])**2
  return sqrt(distance)

#Locate the nearest neighbors
def get_neighbors(train,test_row,num_neightbors):
  distance=[]
  for train_row in train:
    dist = EuclideanDistance(train_row,test_row)
    distance.append((train_row,dist)) #store tuple in list
  distance.sort(key=lambda tup:tup[1])
  neighbors=[]
  for i in range(num_neightbors):
    neighbors.append(distance[i][0])
  return neighbors


# Make a prediction with neighbors
def predict_classification(train, test_row, num_neighbors):
  neighbors = get_neighbors(train, test_row, num_neighbors)
  output_values = [row[-1] for row in neighbors]
  prediction = max(set(output_values), key=output_values.count)
  return prediction

# kNN Algorithm
def k_nearest_neighbors(train, test, num_neighbors):
	predictions = list()
	for row in test:
		output = predict_classification(train, row, num_neighbors)
		predictions.append(output)
	return(predictions)
 

# Load iris dataset
filename = '/content/gdrive/My Drive//ML Data/Iris.csv'
dataset = load_csv(filename)
print('Loaded data file with {0} rows and {1} columns'.format( len(dataset), len(dataset[0])))
#get rid of header
header =dataset[0]
dataset=dataset[1:]

for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# evaluate algorithm
n_folds = 5
num_neighbors = 5
scores = evaluate_algorithm(dataset, k_nearest_neighbors, n_folds, num_neighbors)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))